In [1]:
# Get the data from the parquet file

from pyspark.sql import SparkSession
# memory issues .. see https://stackoverflow.com/questions/21138751/spark-java-lang-outofmemoryerror-java-heap-space
spark = SparkSession.builder.master("local[*]") \
    .config("spark.executor.memory", "12g") \
    .config("spark.driver.memory", "12g") \
    .config("spark.memory.offHeap.enabled", True).config("spark.memory.offHeap.size","8g") \
    .appName("PropertyData").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark
df = spark.read.parquet("/tmp/spark/tarrant-tx.parquet")

# Property used to format output tables better
# spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/19 16:57:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator

subset = [
    'Total_Value', 'Appraised_Value_Int', 'Central_Heat_Ind_idx', 'Central_Air_Ind_idx', 'Swimming_Pool_Ind_idx'
]

# large DF causes "java.lang.NegativeArraySizeException"
logistic_df = df.select(*subset).limit(1000)

logistic_features = [
    'Total_Value', 'Central_Heat_Ind_idx', 'Central_Air_Ind_idx', 'Swimming_Pool_Ind_idx',
]

model_logistic = LogisticRegression(featuresCol="features", labelCol="Appraised_Value_Int", maxIter=5)

# Train
train_logistic, test_logistic = logistic_df.randomSplit([0.7, 0.3])
assembler = VectorAssembler(inputCols=logistic_features, outputCol='features')
train_logistic = assembler.transform(train_logistic)
test_logistic = assembler.transform(test_logistic)
logistic_trained_model = model_logistic.fit(train_logistic)

# Evaluate
logistic_predictions = logistic_trained_model.transform(test_logistic)
evaluator = BinaryClassificationEvaluator(labelCol='Appraised_Value_Int', rawPredictionCol='prediction')
accuracy = evaluator.evaluate(logistic_predictions)

# Print the accuracy
print(f"Accuracy: {accuracy}")

23/06/19 16:57:14 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
23/06/19 17:03:10 WARN DAGScheduler: Broadcasting large task binary with size 22.8 MiB


Accuracy: 1.0


In [3]:
logistic_predictions.show()

23/06/19 17:09:06 WARN DAGScheduler: Broadcasting large task binary with size 22.8 MiB


+-----------+-------------------+--------------------+-------------------+---------------------+-----------------+--------------------+--------------------+----------+
|Total_Value|Appraised_Value_Int|Central_Heat_Ind_idx|Central_Air_Ind_idx|Swimming_Pool_Ind_idx|         features|       rawPrediction|         probability|prediction|
+-----------+-------------------+--------------------+-------------------+---------------------+-----------------+--------------------+--------------------+----------+
|        1.0|                  1|                 0.0|                0.0|                  0.0|    (4,[0],[1.0])|[-0.0016698163476...|[8.14378380962082...|  161880.0|
|        1.0|                  1|                 0.0|                0.0|                  1.0|[1.0,0.0,0.0,1.0]|[-0.0016302136903...|[1.42234180864383...|  271586.0|
|    15681.0|              15681|                 0.0|                0.0|                  0.0|(4,[0],[15681.0])|[-0.0016735901750...|[1.17742240593027...|  16